In [2]:
import joblib
import pandas as pd
movie_features =  pd.read_csv("movie_features.csv")

In [13]:
model = joblib.load('movie_recommender_model.pkl')

def find_similar_movies(movie_id, num_neighbors=5):
    # Get the movie row based on tmdbId from movie_features
    movie_row = movie_features[movie_features['tmdbId'] == movie_id]
    if movie_row.empty:
        print("Movie id not found.")
        return None

    # Prepare query features by dropping 'title' and 'tmdbId'
    query_features = movie_row.drop(columns=['title', 'tmdbId'])
    
    # Transform the query features using the pipeline's preprocessing step
    query_transformed = model.named_steps['columntransformer'].transform(query_features)
    
    # Retrieve neighbors using the NearestNeighbors model inside the pipeline
    distances, indices = model.named_steps['nearestneighbors'].kneighbors(query_transformed)

    # Remove the queried movie from the neighbors if it's present
    query_index = movie_row.index[0]
    neighbor_indices = [idx for idx in indices[0] if idx != query_index]
    neighbor_indices = neighbor_indices[:num_neighbors]
    
    # Return the similar movies as a list of tuples (tmdbId, title)
    similar_movies = movie_features.iloc[neighbor_indices][['tmdbId', 'title']]
    return list(similar_movies.itertuples(index=False, name=None))

In [ ]:
find_similar_movies(192558, num_neighbors=5)  

[(15457, 'Paris 36 (Faubourg 36)'),
 (187628, 'Shed No Tears (Känn ingen sorg)'),
 (243683, 'Step Up All In'),
 (11518, 'Mohabbatein'),
 (54986, 'Chunhyang')]

: 